#### Nombre: **05_modelo_edad**
#### Autores: _Sergio Cañón  Laiz y Ignacio Ruiz de Zuazu Echavarría_
#### CUNEF
#### 15/01/2021


# Librerías

In [11]:
# importamos librerías 

import numpy as np  # álgebra lineal
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Conv2D, MaxPooling2D, AveragePooling2D, GlobalAveragePooling2D
from tensorflow.keras import utils
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
import glob  # pathnames
import cv2  # lectura y procesamiento de las imagenes

import matplotlib.pyplot as plt  # gráficos
import matplotlib.image as mpimg

import os  # lectura de directorios

import keras 
from keras.layers import *
from keras.models import *
from keras.utils.np_utils import to_categorical
from keras import backend as K
from keras import optimizers
import time #tiempos de ejecución
from random import shuffle
from keras.preprocessing import image
import imageio


from tensorflow.keras.preprocessing import image
import matplotlib.pyplot as plt

from keras.optimizers import SGD


# Importación de datos

In [8]:
categorical_labels_age = np.load(r'..\data\04_data_array\categorical_labels_age.npy')

In [5]:
X = np.load(r'..\data\04_data_array\data_32_32_rgb.npy')

In [6]:
size = len(X)
print("La longuitud del los datos son de:", size)

La longuitud del los datos son de: 24099


Usaremos un 30% de las iamgenes como test y el 70% como train.


In [9]:
#creacion train y test set

(x_train, y_train), (x_test, y_test) = (X[:15008],categorical_labels_age[:15008]) , (X[15008:] , categorical_labels_age[15008:])
(x_valid , y_valid) = (x_test[:7000], y_test[:7000])
(x_test, y_test) = (x_test[7000:], y_test[7000:])

In [10]:
print("Número de imagenes en el train:",x_train.shape[0])
print("Número de imagenes en el test:",x_test.shape[0])

Número de imagenes en el train: 15008
Número de imagenes en el test: 2091


# Model Building

## SGD model

In [13]:
model_sgd_gender = tf.keras.Sequential()

# Primera capa de la CNN. El imput debe ajsutarse al tamañ ode la imagen (32 x 32 y 3 de los colores)

model_sgd_age.add(tf.keras.layers.Conv2D(filters=256, kernel_size=2, padding='same', activation='relu', input_shape=(32,32,3))) 
model_sgd_age.add(tf.keras.layers.MaxPooling2D(pool_size=2))
model_sgd_age.add(tf.keras.layers.Dropout(0.3))
model_sgd_age.add(tf.keras.layers.Dense(128, activation='relu')) 

model_sgd_age.add(tf.keras.layers.Conv2D(filters=50, kernel_size=2, padding='same', activation='relu')) 
model_sgd_age.add(tf.keras.layers.MaxPooling2D(pool_size=2))

# drop out= eliminamos el 30% de las imagenes para evitar overfitting
model_sgd_age.add(tf.keras.layers.Dropout(0.3))

model_sgd_age.add(tf.keras.layers.Flatten())
model_sgd_age.add(tf.keras.layers.Dense(256, activation='relu'))

# drop out= eliminamos la mitad de las imagenes para evitar overfitting
model_sgd_age.add(tf.keras.layers.Dropout(0.5)) 


# nuestro data set tiene 5 opciones de raza luego la última capa = 5

model_sgd_age.add(tf.keras.layers.Dense(5, activation='softmax')) 


# Resumen del modelo
model_sgd_age.summary()

model_sgd_age.compile(loss='categorical_crossentropy',
             optimizer=SGD(),
             metrics=['accuracy','AUC','categorical_accuracy'])

h_model_sgd_age = model_sgd_gender.fit(x_train,
         y_train,
         batch_size=100,
         epochs=25,
         validation_data=(x_valid, y_valid))

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 32, 32, 256)       3328      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 16, 16, 256)       0         
_________________________________________________________________
dropout (Dropout)            (None, 16, 16, 256)       0         
_________________________________________________________________
dense (Dense)                (None, 16, 16, 128)       32896     
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 16, 16, 50)        25650     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 8, 8, 50)          0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 8, 8, 50)          0

In [14]:
os.chdir('C:\\Users\\Apuntes\\Desktop\\labUTKfaces\\models\\')

In [17]:
import json
model_sgd_age_history = h_model_sgd_age.history
json.dump(model_sgd_age_history, open("../models/model_sgd_age_history.json", 'w'))

In [18]:
model_sgd_age.save('../models/model_sgd_age_rgb.h')

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: ../models/model_sgd_age_rgb.h\assets


In [19]:
np.save('x_test_edad.npy', x_test)
np.save('y_test_edad.npy', y_test)

# Adam Model

In [20]:
model_adam_age = tf.keras.Sequential()

# Primera capa de la CNN. El imput debe ajsutarse al tamañ ode la imagen (32 x 32 y 3 de los colores)

# aumentamos los filters a 256 y el kernel l obajamos a 2 debido a que la edad es más dificil de predecir

model_adam_age.add(tf.keras.layers.Conv2D(filters=256, kernel_size=2, padding='same', activation='relu', input_shape=(32,32,3))) 
model_adam_age.add(tf.keras.layers.MaxPooling2D(pool_size=2))
model_adam_age.add(tf.keras.layers.Dropout(0.3))
model_adam_age.add(tf.keras.layers.Dense(128, activation='relu'))

model_adam_age.add(tf.keras.layers.Conv2D(filters=256, kernel_size=2, padding='same', activation='relu')) 
model_adam_age.add(tf.keras.layers.MaxPooling2D(pool_size=2))
model_adam_age.add(tf.keras.layers.Dropout(0.3))

model_adam_age.add(tf.keras.layers.Flatten())
model_adam_age.add(tf.keras.layers.Dense(128, activation='relu'))
model_adam_age.add(tf.keras.layers.Dropout(0.5))

model_adam_age.add(tf.keras.layers.Dense(5, activation='softmax'))

# Take a look at the model summary
model_adam_age.summary()

model_adam_age.compile(loss='categorical_crossentropy',
             optimizer='adam',
             metrics=['accuracy','AUC','categorical_accuracy'])

h_model_adam_age = model_adam_age.fit(x_train,
         y_train,
         batch_size=100,
         epochs=25,
         validation_data=(x_valid, y_valid))

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            (None, 32, 32, 256)       3328      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 16, 16, 256)       0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 16, 16, 256)       0         
_________________________________________________________________
dense_3 (Dense)              (None, 16, 16, 128)       32896     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 16, 16, 256)       131328    
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 8, 8, 256)         0         
_________________________________________________________________
dropout_4 (Dropout)          (None, 8, 8, 256)        

In [21]:
model_adam_age_history = h_model_adam_age.history
json.dump(model_adam_age_history, open("../models/model_adam_age_history.json", 'w'))

In [22]:
model_adam_age.save('../models/model_adam_age.h')

INFO:tensorflow:Assets written to: ../models/model_adam_age.h\assets
